**Importing Packages & Loading Data**

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns

In [2]:
visit = pd.read_csv('/content/VisitorLogsData.csv')
user = pd.read_csv('/content/userTable.csv')

In [3]:
visit.head()

,webClientID,VisitDateTime,ProductID,UserID,Activity,Browser,OS,City,Country
0,WI10000050298,2018-05-07 04:28:45.970,pr100631,NaN,NaN,Chrome Mobile,Android,Chennai,India
1,WI10000025922,2018-05-13 07:26:04.964,pr100707,NaN,NaN,Chrome,Windows,NaN,Taiwan
2,WI100000204522,2018-05-11 11:43:42.832,pr100030,NaN,click,Chrome,windows,Gurgaon,India
3,WI10000011974,2018-05-13 15:20:23.436,Pr100192,NaN,CLICK,Chrome,Windows,,
4,WI100000441953,2018-05-08 20:44:25.238,Pr100762,NaN,click,Chrome,mac os x,Iselin,United States


 '**Dropping NA values from UserID**'

In [4]:
df = visit.copy()
df.drop(['Browser', 'City', 'Country'], axis = 1, inplace =True) # these features aren't important
df.dropna(subset=['UserID'], inplace=True)

 **Converting object type features to Upper case**

In [5]:
df['ProductID']= df['ProductID'].str.upper()
df['OS'] = df['OS'].str.upper()
df['Activity'] = df['Activity'].str.upper()

**Converting into single datetime format. i.e Unix value to standard datetime format**

In [6]:
df1 = df.copy()

In [7]:
s,d = df1.shape
print(s)

513069


In [8]:
df1['VisitDateTime'] = df1['VisitDateTime'].replace(np.nan,'NAN')

In [9]:
df1.reset_index(inplace=True)

In [10]:
from numpy import nan
Time = []
for i in range(0, s):
  if ' ' in df1['VisitDateTime'][i]:
    Time.append(pd.to_datetime(df1['VisitDateTime'][i]))
  elif df1['VisitDateTime'][i] == 'NAN':
    Time.append(nan)
    continue
  else:
    Time.append(pd.to_datetime(int(df1['VisitDateTime'][i])))
    #print('else')

In [11]:
df1['Time'] = Time

In [12]:
df1.head(5)

,index,webClientID,VisitDateTime,ProductID,UserID,Activity,OS,Time
0,14,WI100000112772,1526366895249000000,PR100017,U106593,CLICK,ANDROID,2018-05-15 06:48:15.249
1,21,WI1000009977,1527058921790000000,PR101008,U108297,NaN,ANDROID,2018-05-23 07:02:01.790
2,23,WI100000223826,2018-05-10 06:28:53.391,PR100241,U132443,NaN,WINDOWS,2018-05-10 06:28:53.391
3,24,WI10000021998,2018-05-08 12:40:02.153,PR100495,U134616,CLICK,WINDOWS,2018-05-08 12:40:02.153
4,33,WI10000036281,2018-05-11 15:35:43.689,PR100363,U130784,CLICK,CHROME OS,2018-05-11 15:35:43.689


**Calculating last 7 day visit, by the User**

In [13]:
df1.sort_values(['UserID', 'VisitDateTime', 'webClientID'], inplace= True)
df1.reset_index(inplace=True)
df1.drop(['level_0', 'index'], inplace=True, axis = 1)
df1['Time'] = np.where(df1['Time'].notnull(), df1['Time'], df1['Time'].ffill().bfill())
day7 = max(df1['Time']) - pd.to_timedelta(7, unit = 'd') 
no_of_visit_in_7 = df1[df1['Time']>=day7]
visited_in_7_days = no_of_visit_in_7.groupby('UserID')['Time'].nunique()

**Number of Product viewed in last 15 days**

In [14]:
day15 = max(df1['Time']) - pd.to_timedelta(15, unit = 'd')
no_of_visit_in_15 = df1[df1['Time']>=day15] 
no_of_visit_in_15 = pd.DataFrame(no_of_visit_in_15.groupby('UserID')['ProductID'].count())
no_of_visit_in_15.rename(columns={'ProductID': 'no_of_visit_in_15 days'}, inplace=True)

**Calculating Vintage from Signup Date**

In [15]:
user['Signup Date'] = user['Signup Date'].apply(lambda x : x.split(' ')[0])
user['Signup Date'] = pd.to_datetime(user['Signup Date'])
max_time = max(df1['Time'])
user['vin'] = user['Signup Date'].apply(lambda x : max_time - x)
user['vin'] = user['vin'].apply(lambda x : x.days) 
vintage_in_days = user[['UserID', 'vin']]

**Most used product in last 15 days. If multiple product have similar pageload then choose the latest one. if no product viewed in 15 days put Product101**

In [ ]:
df1['day_t_f'] = df1['Time']>=day15
a = df1[df1['day_t_f'] == True]
b = df1[df1['day_t_f'] == False]
b['ProductID'] = 'Product101'
c = a.append(b)


In [17]:
product_in_15_days = c.groupby('UserID')['ProductID'].\
                                              agg([('ProductID','value_counts')])
l1 = len(product_in_15_days)
for i in range(0,10):
  u = pd.DataFrame(product_in_15_days.index.get_level_values(0))
  p = pd.DataFrame(product_in_15_days.index.get_level_values(1))
u['prod'] = p

**Most frequent OS used by User**

In [18]:
df1['OS'] = df1['OS'].ffill().bfill()
most = df1.groupby(['UserID','OS'])['OS'].count()
most_freq_os = pd.DataFrame(most.index.get_level_values(0))
most_freq_os['OS'] = most.index.get_level_values(1)

**Pageload & Click count, in last 7 days, by the User**

In [ ]:
page = df1[df1['Time']>= day7]
page['fill_activity'] = np.where(page['Activity'].notnull(), page['Activity'], page['Activity'].ffill().bfill())
page.drop(['fill_activity'], axis = 1, inplace=True)
pageload_count = page[page['Activity']=='PAGELOAD'].groupby('UserID')['Activity'].count()
click_count = page[page['Activity']=='CLICK'].groupby('UserID')['Activity'].count()

**Joining all inputs**

In [20]:
df_join = user[['UserID']]
final= df_join\
               .merge(visited_in_7_days, how= 'left', on= 'UserID')\
               .merge(no_of_visit_in_15, how= 'left', on= 'UserID')\
               .merge(vintage_in_days, how= 'left', on= 'UserID')\
               .merge(u, how= 'left', on= 'UserID')\
               .merge(most_freq_os, how= 'left', on= 'UserID')\
               .merge(pageload_count, how= 'left', on= 'UserID')\
               .merge(click_count, how= 'left', on= 'UserID')

In [21]:
final.rename(columns = {'Time':'no_of_visit_in_7days', 'vin':'Vintage', 'prod':'Product', 'Activity_x':'pageload_count in7days', 'Activity_y':'click_count in7days'}, inplace=True)

In [22]:
final['no_of_visit_in_7days'].replace(np.nan, '0', inplace=True)
final['no_of_visit_in_15 days'].replace(np.nan, '0', inplace=True)
final['Product'].replace(np.nan, 'Product101', inplace=True)
final['OS'].replace(np.nan, 'OS not specified', inplace=True)
final['click_count in7days'].replace(np.nan, '0', inplace=True)
final['pageload_count in7days'].replace(np.nan, '0', inplace=True)

In [23]:
final

,UserID,no_of_visit_in_7days,no_of_visit_in_15 days,Vintage,Product,OS,pageload_count in7days,click_count in7days
0,U133159,0,0,43,Product101,ANDROID,0,0
1,U129368,0,2,176,PR100071,WINDOWS,0,0
2,U109654,0,2,1895,PR100102,WINDOWS,0,0
3,U109654,0,2,1895,PR101997,WINDOWS,0,0
4,U108998,2,2,129,PR100527,WINDOWS,1,1
...,...,...,...,...,...,...,...,...
109499,U134073,0,0,69,Product101,OS not specified,0,0
109500,U113667,1,1,88,PR100397,MAC OS X,0,1
109501,U128470,0,1,84,PR100219,WINDOWS,0,0
109502,U104005,1,1,125,PR100325,ANDROID,1,0


**Saving the File**

In [ ]:
final.to_csv('rohit.csv', index=False)